In [104]:
!pip install pandas requests

In [105]:
pip install ntscraper

Note: you may need to restart the kernel to use updated packages.


In [106]:
!pip install firebase-admin


In [107]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [108]:
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import os
import json
import firebase_admin
from firebase_admin import credentials, firestore, db
from bs4 import BeautifulSoup
from ntscraper import Nitter

In [109]:
service_account_path = os.getenv('FIREBASE_SERVICE_ACCOUNT_KEY')

In [110]:
if not firebase_admin._apps:
    cred = credentials.Certificate(service_account_path)
    firebase_admin.initialize_app(cred)

In [111]:
db = firestore.client()

In [112]:
scraper = Nitter()
scraper.working_instances = ['https://nitter.net']

Testing instances: 100%|██████████| 16/16 [00:19<00:00,  1.24s/it]


In [113]:
def getTweets(name, modes, num):
    total = scraper.get_tweets(name, mode=modes, number=num)
    print(total)
    
    final_tweets = []
    if 'tweets' in total:
        for tweet in total['tweets']:  # Adjust the key to match actual data
            data = [tweet['link'], tweet['text'], tweet['date']]
            final_tweets.append(data)
        data = pd.DataFrame(final_tweets, columns=['link', 'text', 'date'])
        return data
    else:
        print("Key 'tweets' not found in response")
        return None

In [127]:
data = getTweets('mpox', 'term', 25)
print(data)

22-Sep-24 15:41:24 - No instance specified, using random instance https://nitter.lucabased.xyz
22-Sep-24 15:41:25 - Fetching error: Instance has been rate limited.Use another instance or try again later.
{'tweets': [], 'threads': []}
Empty DataFrame
Columns: [link, text, date]
Index: []


In [115]:
# Function to read existing JSON data (if any)
def read_existing_json(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            try:
                data = json.load(file)
                return data
            except json.JSONDecodeError:
                return {}
    else:
        return {}

# Function to append data to JSON without duplicates
def append_to_json(new_data, file_path):
    existing_data = read_existing_json(file_path)
    
    # Create a set of existing tweet links (to identify duplicates)
    existing_links = {tweet['link'] for tweet in existing_data.values()}
    
    # Append only new tweets (those not already in the existing JSON file)
    for i, tweet in enumerate(new_data):
        if tweet['link'] not in existing_links:
            existing_data[str(len(existing_data))] = tweet  # Use str(len) as the new index for appending
    
    # Write the updated data back to the JSON file
    with open(file_path, 'w') as file:
        json.dump(existing_data, file, indent=4)
    
    print("Data appended to JSON file successfully!")

In [128]:
df = pd.DataFrame(data[['link', 'date', 'text']])
tweets_dict = df.to_dict(orient='records')
append_to_json(tweets_dict, 'tweets_filtered.json')
tweets_dict

Data appended to JSON file successfully!


[]

In [123]:
def upload_to_firebase(data):
    collection_ref = db.collection('tweets')

    #records = data.to_dict(orient='records')
    
    for tweet in data:
        tweet_link = tweet["link"]
        existing_tweets = collection_ref.where('link', '==', tweet_link).stream()
        if not any(existing_tweets):
            doc_ref = collection_ref.document() 
            doc_ref.set(tweet)
            print(f"Uploaded tweet with ID {tweet_link}")
        else:
            print(f"Duplicate tweet with ID {tweet_link} skipped.")
    print("Data uploaded to Firebase successfully!")

In [125]:
upload_to_firebase(tweets_dict)

Uploaded tweet with ID https://twitter.com/david_r_morgan/status/1837685111317270605#m
Uploaded tweet with ID https://twitter.com/Globalbiosec/status/1837942883493859510#m
Uploaded tweet with ID https://twitter.com/toobaffled/status/1837660701084864897#m
Uploaded tweet with ID https://twitter.com/rtncofficielle1/status/1837611704714240495#m
Uploaded tweet with ID https://twitter.com/newstart_2024/status/1837845241518227963#m
Uploaded tweet with ID https://twitter.com/apostolo21/status/1837891275984396693#m
Uploaded tweet with ID https://twitter.com/Attwhoredasilva/status/1837872291083849826#m
Uploaded tweet with ID https://twitter.com/EliseAndre43768/status/1837951409167257898#m
Uploaded tweet with ID https://twitter.com/orangotango6942/status/1837951366930878507#m
Uploaded tweet with ID https://twitter.com/YisusBell/status/1837950987018846615#m
Uploaded tweet with ID https://twitter.com/eftwyrd/status/1837950697742258572#m
Uploaded tweet with ID https://twitter.com/DerAchsenZeit/statu